In [4]:
import random
from sqlalchemy.orm import Session
from datetime import datetime, timedelta
from typing import Dict, Any, List
from pydantic import BaseModel, ValidationError
from google import genai
from google.genai import types
import json
import time
import os
import random
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio

load_dotenv()



class Theme(BaseModel):
    title: str
    story: str

class ThemeGenerate(BaseModel):
    themes: List[Theme]

def generate_theme_title_and_story(campaign_title: str, insight: str, description: str, target_customer:str) -> List[tuple[str, str]]:
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    
    # Generate response using Gemini API (synchronous version)
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=f"Tạo 5 thương hiệu cho pages với các thông tin {insight} {target_customer}. Mỗi thương hiệu phải có title và story khác nhau. Phải viết bằng tiếng việt",
        config=types.GenerateContentConfig(
            response_mime_type='application/json',
            response_schema=ThemeGenerate,
            system_instruction=types.Part.from_text(text=description),
        ),
    )
    
    # Extract the response
    print("Generated 5 themes based on user prompt.")
    content = json.loads(response.text)
    
    # Validate and parse the response using Pydantic
    themes_data = ThemeGenerate(**content)
    
    # Convert list of themes to list of tuples using dot notation for Pydantic model attributes
    return [(theme.title, theme.story) for theme in themes_data.themes]

In [5]:
generate_theme_title_and_story(campaign_title="xin chao", \
    insight="123", description="123", target_customer="123")


Generated 5 themes based on user prompt.


[('Hương Vị Quê Nhà',
  'Thương hiệu mang đậm nét truyền thống, gợi nhớ những món ăn dân dã, nguyên liệu tươi ngon từ khắp vùng miền Việt Nam. Mỗi sản phẩm là một câu chuyện về văn hóa và con người.'),
 ('Sức Khỏe Vàng',
  'Dòng sản phẩm chăm sóc sức khỏe toàn diện, từ thực phẩm bổ dưỡng đến các liệu pháp thư giãn. Chúng tôi cam kết mang đến cho bạn một cuộc sống khỏe mạnh và tràn đầy năng lượng.'),
 ('Phong Cách Tinh Tế',
  'Thương hiệu thời trang dành cho những người yêu thích sự thanh lịch và sang trọng. Thiết kế độc đáo, chất liệu cao cấp, thể hiện đẳng cấp và gu thẩm mỹ riêng biệt.'),
 ('Khám Phá Thế Giới',
  'Dịch vụ du lịch trọn gói, đưa bạn đến những vùng đất mới lạ, trải nghiệm những nền văn hóa độc đáo. Chúng tôi sẽ đồng hành cùng bạn trên mọi hành trình khám phá.'),
 ('Ngôi Nhà Hạnh Phúc',
  'Cung cấp các sản phẩm nội thất và trang trí nhà cửa, giúp bạn tạo nên một không gian sống ấm cúng, tiện nghi và đầy phong cách. Nơi bạn tìm thấy sự bình yên và hạnh phúc.')]